In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

In [2]:
import _pickle as pickle
import random

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.set_device(1)

In [5]:
%run Model.ipynb

In [6]:
emb = pickle.load(open('./Meta/glove_100d_vectors', 'rb'))
emb.append(np.zeros(100))
emb = torch.Tensor(emb)

In [7]:
urls = pickle.load(open('./Meta/urls', 'rb'))
no_urls = len(urls)

In [8]:
class_mapping = {
    'left': 0,
    'left-center': 1,
    'least':2,
    'right':3,
    'right-center':4,
}

In [9]:
def pad_data(data, max_sen_len=50, max_doc_len=90, padding_idx=400003):
    lines = []
    lengths = []
    for doc in data:
        lines.append([])
        lengths.append([])
        for line in doc[2][0:max_doc_len]:
            if len(line) > max_sen_len:
                line = line[0:max_sen_len]
                lengths[-1].append(max_sen_len)
            else:
                lengths[-1].append(len(line))
                line = line + [padding_idx for _ in range(max_sen_len - len(line))]
            lines[-1].append(line)
    return lines, lengths

In [10]:
def pad_titles(data, max_sen_len=20, padding_idx=400003):
    lines = []
    for row in data:
        line = row[0]
        if len(line) > max_sen_len:
            lines.append(line[0:max_sen_len])
        else:
            lines.append(line + [padding_idx for _ in range(max_sen_len - len(line))])
    return lines

In [11]:
class TrainModel():
    def __init__(self, model):
        self.model = model
        self.raw_data, self.lines, self.pad_lengths, self.truth, self.class_, self.urls = None, None, None, None, None, None
        self.file_no = 0
        self.max_file_no = 8
        self.batch_size = 20
        
        self.loss1 = nn.BCEWithLogitsLoss()
        self.loss2 = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters())
        self.losses = []
        
    def __get_next_batch(self):
        pass
    
    def __load_next(self):
        self.raw_data = pickle.load(open('training_' + str(self.file_no), 'rb'))
        self.lines, self.pad_lengths = pad_data(self.raw_data)
        self.class_ = [class_mapping[row[-1]] for row in self.raw_data]
        self.truth = [int(row[-2]) for row in self.raw_data]
        self.lengths = [[] for _ in range(90)]
        for i, j in enumerate(self.lines):
            if not len(j) == 0:
                self.lengths[len(j) - 1].append(i)
        self.urls = [i[1][0:90] for i in self.raw_data]
        self.batches = []
        for unit in self.lengths:
            for i in range(0, len(unit), self.batch_size):
                self.batches.append(unit[i:i+self.batch_size])
        self.titles = pad_titles(self.raw_data)
        random.shuffle(self.batches)
                
    def train_epoch(self):
       # while self.file_no < self.max_file_no:
        self.__load_next()
        for batch in self.batches:
            if len(batch) == 20:
                self.model.zero_grad()
                input = torch.cuda.LongTensor([self.lines[i] for i in batch])
                urls = torch.cuda.LongTensor([self.urls[i] for i in batch])
                titles = torch.cuda.LongTensor([self.titles[i] for i in batch])
                truth = torch.cuda.FloatTensor([[self.truth[i]] for i in batch])
                pbias, ptruth = self.model(input, urls, titles)
                loss = self.loss2(ptruth, truth)
                print(str(self.batches.index(batch)) + ' ' + str(loss), end='\r')
                self.losses.append(loss.cpu().data)
                loss.backward()
                self.optimizer.step()

In [12]:
m = Model(no_urls, emb)
m.cuda()

Model(
  (sentenceEncoder): LSTMSentenceEncoderParallel(
    (embeddings): Embedding(400004, 100, padding_idx=400003)
    (sentenceEncoder): LSTM(100, 150, batch_first=True, bidirectional=True)
  )
  (sourceBias): SourceBiasParallel(
    (trans): Linear(in_features=300, out_features=300, bias=True)
    (source_embeddings): Embedding(210381, 300)
  )
  (documentEncoder): LSTM(300, 300, batch_first=True, bidirectional=True)
  (documentAttention): Attention(
    (trans): Bilinear(in1_features=600, in2_features=300, out_features=1, bias=True)
  )
  (biasMLP): MLP(
    (layer1): Linear(in_features=600, out_features=300, bias=True)
    (layer2): Linear(in_features=300, out_features=5, bias=True)
  )
  (truthMLP): MLP(
    (layer1): Linear(in_features=600, out_features=300, bias=True)
    (layer2): Linear(in_features=300, out_features=1, bias=True)
  )
)

In [13]:
t = TrainModel(m)

In [ ]:
t.train_epoch()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(t.losses)